<a href="https://colab.research.google.com/github/Junjaee/Project_PinocchiOH/blob/main/Member/SHIN/Naver_Movie_Crawling_v1.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import re
import os
from random import randrange

import requests
from bs4 import BeautifulSoup as bs
import openpyxl
from urllib.request import urlretrieve

In [7]:
def crawling(start_code, finish_code):
    
    j = 0
    wb = openpyxl.Workbook()
    sheet = wb.active
    sheet.append(["MOVIE_CODE","MOVIE_TITLE","MOVIE_YEAR","MOVIE_RATING", "MOVIE_GENRE", "MOVIE_COUNTRY", "MOVIE_RTIME", "MOVIE_DIRECTOR", "MOVIE_ACTOR", 
                  "MOVIE_AGE10", "MOVIE_AGE20", "MOVIE_AGE30", "MOVIE_AGE40", "MOVIE_AGE50"])

    global is_ok
    is_ok = False

    for i in range(start_code, finish_code):

        MOVIE_CODE = str(i)
        raw = requests.get("https://movie.naver.com/movie/bi/mi/basic.naver?code=" + MOVIE_CODE)
        html = bs(raw.content, 'html.parser')
        movie = html.select("div.article")    
        
        try:
            for a, m in enumerate(movie):
                # print(a, m)
                # print("*"*40)

                # (3-1) 영화제목 수집
                title = m.select_one("h3.h_movie a")
                MOVIE_TITLE = title.text
                print(MOVIE_TITLE)

                # (3-2) 영화 개봉년도 수집
                rdate = m.select("strong.h_movie2")
                MOVIE_YEAR = rdate[-1].text[-4:]
                print(MOVIE_YEAR)                

                # (3-3) 영화평점 수집
                score = m.select("div.main_score div.score.score_left div.star_score a em")
                scores = []
                for s in score:
                    scores.append(s.text)                
                MOVIE_RATING = ''.join(scores)
                print(MOVIE_RATING)       

                # (3-4) 영화장르 수집
                GENRE = m.select("dl.info_spec dd:nth-of-type(1) p span:nth-of-type(1) a")
                for g in GENRE:
                    print(g.text)

                # (3-5) 영화 생산 국가
                COUNTRY = m.select("dl.info_spec dd:nth-of-type(1) p span:nth-of-type(2) a")
                for c in COUNTRY:
                    print(c.text)

                # (3-6) 영화 상영시간 수집
                rtime = m.select_one("dl.info_spec dd p span:nth-of-type(3)")
                MOVIE_RTIME = rtime.text[:-2] # 분 생략하고 숫자만 출력
                print(MOVIE_RTIME)

                # (3-7) 영화감독 수집
                DIRECTOR = m.select("dl.info_spec dd:nth-of-type(2) p a")
                for d in DIRECTOR:
                    print(d.text)

                # (3-8) 영화배우 수집
                ACTOR = m.select("dl.info_spec dd:nth-of-type(3) p a")
                for a in ACTOR:
                    print(a.text)

                # #(3-9) 성별 선호도(남자)
                # male = html.select("#content > div.article > div.mv_info_area > div.mv_info > div.viewing_graph")
                # # print(male)
                # for a in male:
                #     print(a.text)

                # (3-10-1) 연령별 선호도(10대)
                age_ten = m.select("div.bar_graph div:nth-of-type(1) strong")
                MOVIE_AGE10 = age_ten[0].text[:-1]
                print(MOVIE_AGE10)

                # (3-10-2) 연령별 선호도(20대)
                age_twenty = m.select("div.bar_graph div:nth-of-type(3) strong")
                MOVIE_AGE20 = age_twenty[0].text[:-1]
                print(MOVIE_AGE20)

                # (3-10-3) 연령별 선호도(30대)
                age_thirty = m.select("div.bar_graph div:nth-of-type(5) strong")
                MOVIE_AGE30 = age_thirty[0].text[:-1]
                print(MOVIE_AGE30)

                # (3-10-4) 연령별 선호도(40대)
                age_fourty = m.select("div.bar_graph div:nth-of-type(7) strong")
                MOVIE_AGE40 = age_fourty[0].text[:-1]
                print(MOVIE_AGE40)

                # (3-10-5) 연령별 선호도(50대 이상)
                age_fifty = m.select("div.bar_graph div:nth-of-type(9) strong")
                MOVIE_AGE50 = age_fifty[0].text[:-1]
                print(MOVIE_AGE50)
                print(len(MOVIE_AGE50))

                if len(MOVIE_AGE50) > 0:
                    # (7) 영화관련정보 엑셀(xlsx) 형식 저장
                    # (7-1) 데이터 만들기-1 : HTML로 가져온 영화장르/영화감독/영화배우 정보에서 TEXT정보만 뽑아서 리스트 형태로 만들기
                    GENREs = [g.text for g in GENRE]
                    COUNTRYs = [c.text for c in COUNTRY]
                    DIRECTORs = [d.text for d in DIRECTOR]
                    ACTORs = [a.text for a in ACTOR]

                    #(7-2) 데이터 만들기-2 : 여러 개로 이루어진 리스트 형태를 하나의 문자열 형태로 만들기
                    MOVIE_GENRE = ','.join(GENREs)
                    MOVIE_COUNTRY = ','.join(COUNTRYs)
                    MOVIE_DIRECTOR = ','.join(DIRECTORs)
                    MOVIE_ACTOR = ','.join(ACTORs)

                    # (7-3) 영화관련정보 엑셀 행 추가 : line by line 으로 추가하기
                    sheet.append([MOVIE_CODE,MOVIE_TITLE,MOVIE_YEAR,MOVIE_RATING, MOVIE_GENRE, MOVIE_COUNTRY, MOVIE_RTIME, MOVIE_DIRECTOR, MOVIE_ACTOR, 
                                MOVIE_AGE10, MOVIE_AGE20, MOVIE_AGE30, MOVIE_AGE40, MOVIE_AGE50])

                    is_ok = True

            if is_ok == True:
                j = j + 1
            print(finish_code - start_code, "개중에", finish_code - i - 1, "개 남음")
            print(j, "개의 영화 정보저장 완료")

        except:
            print("에러발생")
            print(finish_code - start_code, "개중에", finish_code - i - 1, "개 남음")
            continue

        finally:
            print("완료")
            print("*"*40)
            wb.save("navermovie.xlsx")

In [ ]:
crawling(10001, 210000)

시네마 천국
1988
9.33
드라마
멜로/로맨스
프랑스
이탈리아
124
쥬세페 토르나토레
마르코 레오나르디
필립 느와레
자끄 페렝
9
47
25
6
12
2
199999 개중에 199998 개 남음
1 개의 영화 정보저장 완료
완료
****************************************
빽 투 더 퓨쳐
1985
9.42
SF
코미디
미국
120
로버트 저메키스
마이클 J. 폭스
크리스토퍼 로이드
리 톰슨
2
61
18
16
2
1
199999 개중에 199997 개 남음
2 개의 영화 정보저장 완료
완료
****************************************
빽 투 더 퓨쳐 2
1989
9.24
SF
코미디
미국
107
로버트 저메키스
마이클 J. 폭스
크리스토퍼 로이드
0
45
52
3
0
1
199999 개중에 199996 개 남음
3 개의 영화 정보저장 완료
완료
****************************************
빽 투 더 퓨쳐 3
1990
9.12
SF
서부
판타지
코미디
미국
117
로버트 저메키스
마이클 J. 폭스
크리스토퍼 로이드
메리 스틴버겐
에러발생
199999 개중에 199995 개 남음
완료
****************************************
스타워즈 에피소드 4 - 새로운 희망
1977
9.00
SF
모험
액션
판타지
미국
121
조지 루카스
마크 해밀
해리슨 포드
캐리 피셔
에러발생
199999 개중에 199994 개 남음
완료
****************************************
스타워즈 에피소드 5 - 제국의 역습
1980
9.08
SF
모험
액션
판타지
미국
124
어빈 케쉬너
마크 해밀
해리슨 포드
캐리 피셔
에러발생
199999 개중에 199993 개 남음
완료
****************************************
스타워즈 에피소드 6 - 제다이의 귀환
1983
9.00
SF
모험
액션
판타지
미국
133
리